In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AbtBuyBenchmark

benchmark = AbtBuyBenchmark(data_dir_path="../data/")
benchmark

17:39:26 INFO:Extracting Abt-Buy...
17:39:26 INFO:Reading Abt-Buy row_dict...
17:39:26 INFO:Reading Abt-Buy train.csv...
17:39:26 INFO:Reading Abt-Buy valid.csv...
17:39:26 INFO:Reading Abt-Buy test.csv...


<AbtBuyBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Textual/Abt-Buy/abt_buy_exp_data.zip

## Preprocess

In [5]:
attr_list = ['name', 'description', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_name': {
        'source_attr': 'name',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'description': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'semantic_description': {
        'source_attr': 'description',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_list=benchmark.row_dict.values())
row_numericalizer.attr_info_dict

17:39:26 INFO:For attr=name, computing actual max_str_len
17:39:26 INFO:actual_max_str_len=15 must be even to enable NN pooling. Updating to 16
17:39:26 INFO:For attr=name, using actual_max_str_len=16
17:39:26 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:39:30 INFO:For attr=description, computing actual max_str_len
17:39:30 INFO:actual_max_str_len=15 must be even to enable NN pooling. Updating to 16
17:39:30 INFO:For attr=description, using actual_max_str_len=16
17:39:30 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:39:33 INFO:For attr=price, computing actual max_str_len
17:39:33 INFO:actual_max_str_len=7 must be even to enable NN pooling. Updating to 8
17:39:33 INFO:For attr=price, using actual_max_str_len=8


{'name': NumericalizeInfo(source_attr='name', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_name': NumericalizeInfo(source_attr='name', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=50,
    eval_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    row_numericalizer,
    ann_k=ann_k,
    eval_with_clusters=False
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name),
    reload_dataloaders_every_epoch=True
)

17:39:33 INFO:GPU available: True, used: True
17:39:33 INFO:TPU available: None, using: 0 TPU cores
17:39:33 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

17:39:33 INFO:Train pair count: 616
17:39:33 INFO:Valid pair count: 206
17:39:33 INFO:Test pair count: 206
17:39:36 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.7 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
4.9 M     Trainable params
2.7 M     Non-trainable params
7.7 M     Total params


17:39:36 WARNING:true_pair_set has 6 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [15]:
model.blocker_net.get_signature_weights()

{'name': 0.3419387936592102,
 'semantic_name': 0.21766455471515656,
 'description': 0.20726965367794037,
 'semantic_description': 0.14936192333698273,
 'price': 0.08376504480838776}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.3763837638376384,
 'valid_f1_at_0.5': 0.6714285714285715,
 'valid_f1_at_0.7': 0.7746835443037974,
 'valid_f1_at_0.9': 0.3764705882352941,
 'valid_pair_entity_ratio_at_0.3': 2.1310679611650487,
 'valid_pair_entity_ratio_at_0.5': 0.8592233009708737,
 'valid_pair_entity_ratio_at_0.7': 0.4587378640776699,
 'valid_pair_entity_ratio_at_0.9': 0.11893203883495146,
 'valid_precision_at_0.3': 0.23234624145785876,
 'valid_precision_at_0.5': 0.5310734463276836,
 'valid_precision_at_0.7': 0.8095238095238095,
 'valid_precision_at_0.9': 0.9795918367346939,
 'valid_recall_at_0.3': 0.9902912621359223,
 'valid_recall_at_0.5': 0.912621359223301,
 'valid_recall_at_0.7': 0.7427184466019418,
 'valid_recall_at_0.9': 0.23300970873786409}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

17:44:59 INFO:Train pair count: 616
17:44:59 INFO:Valid pair count: 206
17:44:59 INFO:Test pair count: 206


[{'test_f1_at_0.3': 0.4063745019920318,
  'test_f1_at_0.5': 0.7423076923076922,
  'test_f1_at_0.7': 0.8010204081632654,
  'test_f1_at_0.9': 0.4198473282442748,
  'test_pair_entity_ratio_at_0.3': 1.9416058394160585,
  'test_pair_entity_ratio_at_0.5': 0.7639902676399026,
  'test_pair_entity_ratio_at_0.7': 0.45255474452554745,
  'test_pair_entity_ratio_at_0.9': 0.1362530413625304,
  'test_precision_at_0.3': 0.2556390977443609,
  'test_precision_at_0.5': 0.6146496815286624,
  'test_precision_at_0.7': 0.8440860215053764,
  'test_precision_at_0.9': 0.9821428571428571,
  'test_recall_at_0.3': 0.9902912621359223,
  'test_recall_at_0.5': 0.9368932038834952,
  'test_recall_at_0.7': 0.7621359223300971,
  'test_recall_at_0.9': 0.2669902912621359}]